In [1]:
import os
import sys
import pickle as pkl
import pandas as pd

repo_path = '/root/ADA2023/ada-2023-project-adacadabra2023/ada-2023-project-adacadabra2023' # CHANGE
sys.path.append(repo_path)

# from src.data.Data import load_data

In [2]:
dpath = os.path.join(repo_path, 'Data', 'Unified_ratings.pkl')
with open(dpath, 'rb') as f:
    df = pkl.load(f)
print(df.shape)

(10932248, 18)


In [3]:
df.columns

Index(['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'date',
       'rating', 'user_id', 'Procedence', 'country_user', 'country_brewery',
       'year', 'Season', 'z_score', 'exp_mean', 'diff_exp_mean', 'ith_rating',
       'detrend'],
      dtype='object')

In [4]:
filtered_df = df[['style','Season', 'country_brewery']]
filtered_df

sty_country = filtered_df.groupby(['country_brewery', 'Season'], as_index=False).nunique()
sty_country

,country_brewery,Season,style
0,Albania,Autumn,3
1,Albania,Spring,3
2,Albania,Summer,3
3,Albania,Winter,3
4,Andorra,Autumn,1
...,...,...,...
859,Zambia,Winter,1
860,Zimbabwe,Autumn,1
861,Zimbabwe,Spring,1
862,Zimbabwe,Summer,1


In [5]:
select_countries = sty_country.groupby('country_brewery', as_index=False).min()
keep_countries = select_countries[select_countries['style'] >= 3]
valid_countries = keep_countries.country_brewery.values

In [6]:
filtered_countries = filtered_df[filtered_df['country_brewery'].isin(valid_countries)]
filtered_countries

,style,Season,country_brewery
0,Maibock / Helles Bock,Autumn,"United States, Louisiana"
1,Maibock / Helles Bock,Winter,"United States, Louisiana"
2,Maibock / Helles Bock,Spring,"United States, Louisiana"
3,Maibock / Helles Bock,Spring,"United States, Louisiana"
4,Maibock / Helles Bock,Autumn,"United States, Louisiana"
...,...,...,...
10932243,India Pale Ale (IPA),Summer,Poland
10932244,India Pale Ale (IPA),Summer,Poland
10932245,India Pale Ale (IPA),Summer,Poland
10932246,India Pale Ale (IPA),Summer,Poland


In [ ]:
def filter_countries(df, min_styles=3):
    filtered_df = df[['style','Season', 'country_brewery']]

    sty_country = filtered_df.groupby(['country_brewery', 'Season'], as_index=False).nunique()

    select_countries = sty_country.groupby('country_brewery', as_index=False).min()
    keep_countries = select_countries[select_countries['style'] >= min_styles]
    valid_countries = keep_countries.country_brewery.values
    filtered_countries = filtered_df[filtered_df['country_brewery'].isin(valid_countries)]
    return filtered_countries

In [7]:
top_4_beerstyles = filtered_countries.groupby(['country_brewery', 'Season'])['style'].value_counts().groupby(['country_brewery', 'Season']).head(3).reset_index(name='ratings')

styles_combined = top_4_beerstyles.groupby(['country_brewery', 'Season'], as_index= False).agg(lambda x: x.tolist())

In [12]:
styles_combined[['1st_Style', '2nd_Style', '3rd_Style']] = pd.DataFrame(styles_combined['style'].tolist(), index=styles_combined.index)
styles_combined[['1st_Rating', '2nd_Rating', '3rd_Rating']] = pd.DataFrame(styles_combined['ratings'].tolist(), index=styles_combined.index)
styles_combined = styles_combined.drop(['style', 'ratings'], axis=1)


In [13]:
styles_combined

,country_brewery,Season,1st_Style,2nd_Style,3rd_Style,1st_Rating,2nd_Rating,3rd_Rating
0,Albania,Autumn,Pale Lager,Dunkel/Tmavý,Pilsener,61,12,12
1,Albania,Spring,Pale Lager,Pilsener,Dunkel/Tmavý,63,30,16
2,Albania,Summer,Pale Lager,Pilsener,Dunkel/Tmavý,92,36,32
3,Albania,Winter,Pale Lager,Pilsener,Dunkel/Tmavý,48,16,5
4,Argentina,Autumn,Pale Lager,American Adjunct Lager,American Strong Ale,217,89,40
...,...,...,...,...,...,...,...,...
627,Virgin Islands (U.S.),Winter,India Pale Ale (IPA),Amber Ale,Fruit Beer,55,43,36
628,Wales,Autumn,India Pale Ale (IPA),American Amber / Red Ale,Belgian Strong Ale,2539,1542,1455
629,Wales,Spring,American Amber / Red Ale,India Pale Ale (IPA),American Double / Imperial IPA,2876,1733,1305
630,Wales,Summer,India Pale Ale (IPA),American Double / Imperial IPA,American Amber / Red Ale,2716,1711,1627


In [15]:
styles_combined.to_pickle('training_df.pkl')